In [16]:
# !pip install scikit-learn --upgrade

In [146]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector

url = "https://drive.google.com/file/d/11shN9FaBAGPpOmHrjJQkVqMizzjuUuK_/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]


data = pd.read_csv(path)
data

,cap.shape,cap.color,bruises,stalk.color.above.ring,stalk.color.below.ring,population,Id,poisonous
0,k,e,False,w,w,v,6573,1
1,f,e,True,p,w,y,4426,0
2,b,w,False,w,w,s,7018,0
3,k,g,False,w,w,n,5789,0
4,f,n,True,p,g,v,6187,0
...,...,...,...,...,...,...,...,...
6494,f,g,False,b,b,v,7002,1
6495,k,e,False,p,p,v,7323,1
6496,f,g,False,n,b,y,6206,1
6497,f,n,True,w,w,s,4658,0


Preparing Data for Machine Learning
Now we will choose x and y axis and split these datas to train or test.

In [147]:
data.drop('Id',axis=1,inplace=True)
X = data.drop(columns=['poisonous','bruises'])
y = data["poisonous"]

# # Split data for train and test

In [148]:
  
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)   

In [178]:
X_test

,cap.shape,cap.color,stalk.color.above.ring,stalk.color.below.ring,population
5572,f,e,p,w,v
1703,f,g,p,g,y
5862,x,g,n,p,y
1773,f,n,w,w,a
5709,k,n,p,p,v
...,...,...,...,...,...
4778,f,e,w,p,y
3494,k,n,w,w,v
624,f,g,b,p,y
69,x,n,p,w,y


In [125]:
X_train.shape , X_test.shape

((5199, 5), (1300, 5))

In [126]:
y_train.shape , y_test.shape

((5199,), (1300,))

In [152]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# numeric_pipe = make_pipeline(
#     SimpleImputer(strategy='mean') 

# )   
cat_col= list(X_train.select_dtypes(include=["object"]))
categorical_pipeline = Pipeline(steps=[
    ('one-hot', OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

full_processor = ColumnTransformer(transformers=[
    ('category', categorical_pipeline, cat_col) ])
 
#     categorical_pipe_onehot = make_pipeline(
#     (SimpleImputer(strategy='constant',fill_value='NaN')),
#     (OneHotEncoder(handle_unknown='ignore',sparse_output=False)))


# preprocessor = make_column_transformer(
#           (numeric_pipe,make_column_selector(dtype_include='number')),
#           (categorical_pipe_onehot,make_column_selector(dtype_include='object'))

# )

pipe = make_pipeline(full_processor, 
                       DecisionTreeClassifier()).set_output(transform='pandas')

pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('category',
                                                  Pipeline(steps=[('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['cap.shape', 'cap.color',
                                                   'stalk.color.above.ring',
                                                   'stalk.color.below.ring',
                                                   'population'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [106]:
full_processor

ColumnTransformer(transformers=[('category',
                                 Pipeline(steps=[('one-hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cap.shape', 'cap.color',
                                  'stalk.color.above.ring',
                                  'stalk.color.below.ring', 'population'])])

In [154]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('category',
                                                  Pipeline(steps=[('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['cap.shape', 'cap.color',
                                                   'stalk.color.above.ring',
                                                   'stalk.color.below.ring',
                                                   'population'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [160]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'decisiontreeclassifier__max_depth': range(2, 12),
    'decisiontreeclassifier__min_samples_leaf': range(3, 10, 2),
    'decisiontreeclassifier__min_samples_split': range(3, 40, 5),
    'decisiontreeclassifier__criterion':['gini', 'entropy']
}
grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring='accuracy', 
    cv=5,
    verbose=1
)
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('category',
                                                                         Pipeline(steps=[('one-hot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['cap.shape',
                                                                          'cap.color',
                                                                          'stalk.color.above.ring',
                                                                          'stalk.color.below.ring',
                                                                          'population'])])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 12),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 10, 2),
                         'decisiontreeclassifier__min_samples_split': range(3, 40, 5)},
             scoring='accuracy', verbose=1)

In [161]:
# best parameters
grid_search.best_params_

{'decisiontreeclassifier__criterion': 'gini',
 'decisiontreeclassifier__max_depth': 11,
 'decisiontreeclassifier__min_samples_leaf': 3,
 'decisiontreeclassifier__min_samples_split': 3}

In [162]:
# cross validation average accuracy
grid_search.best_score_

0.8995970607832975

In [163]:
grid_search.best_estimator_.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('category',
                                                  Pipeline(steps=[('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['cap.shape', 'cap.color',
                                                   'stalk.color.above.ring',
                                                   'stalk.color.below.ring',
                                                   'population'])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=11, min_samples_leaf=3,
                                        min_samples_split=3))])

In [164]:
pipe.predict(X_train)

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [165]:
predictions = pipe.predict(X_test)

In [166]:
predictions

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [142]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import  f1_score

In [179]:
# Train
y_pred = predictions


In [180]:
y_pred 

array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

In [167]:

url = "https://drive.google.com/file/d/1FYnZsiRQTI4KEm-V6K9IzfXvLEJllRDL/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]


data_new = pd.read_csv(path)
data_new

,Id,cap.shape,cap.color,stalk.color.above.ring,stalk.color.below.ring,bruises,population
0,5165,f,w,w,w,True,v
1,4281,f,g,n,p,False,v
2,231,x,n,w,p,True,v
3,3890,f,n,w,w,False,s
4,1521,x,y,n,p,False,v
...,...,...,...,...,...,...,...
1620,983,x,n,w,y,False,v
1621,7055,x,e,p,g,True,v
1622,7456,x,y,b,n,False,y
1623,5132,x,w,w,w,True,n


In [168]:
predictions = grid_search.best_estimator_.predict(data_new)

In [169]:
id_column = data_new.pop('Id')

In [170]:
results = pd.DataFrame({'Id':id_column,'poisonous':predictions})


In [171]:
results.to_csv('results_mushro.csv',index=False)